In [2]:
import os
import shutil

In [3]:
files = os.listdir("../../data/results")
files

['h3',
 'centers_using_si.csv',
 'h23_short',
 'h6',
 'h4',
 'h24',
 'h1',
 'convolution.png',
 'h16_short',
 'h22_short',
 'using_ap.png',
 'h13_short',
 'h11_short',
 'h20_short',
 'x_y_using_si.png',
 'h17_short',
 'h10_short',
 'convolution.csv',
 'h1_try',
 'centers_using_dc.csv',
 'using_dc.png',
 'h18_short',
 'h15_short',
 'h9_short',
 'h21_short',
 'h7',
 'h7_short',
 'ref',
 'h14_short',
 'h8_short',
 'x_y_using_dc.png',
 'new.csv',
 'h12_short',
 'ref.txt',
 'h25_short',
 'x_y_using_ap.png',
 'centers_using_ap.csv',
 'h2',
 'h19_short',
 'using_si.png',
 'h5_short']

In [4]:
folders = [f for f in files if "." not in f]
folders

['h3',
 'h23_short',
 'h6',
 'h4',
 'h24',
 'h1',
 'h16_short',
 'h22_short',
 'h13_short',
 'h11_short',
 'h20_short',
 'h17_short',
 'h10_short',
 'h1_try',
 'h18_short',
 'h15_short',
 'h9_short',
 'h21_short',
 'h7',
 'h7_short',
 'ref',
 'h14_short',
 'h8_short',
 'h12_short',
 'h25_short',
 'h2',
 'h19_short',
 'h5_short']

In [6]:
original_file = "Ref_Notebook.ipynb"
folders = [
    "h14_short",
    "h15_short",
    "h16_short",
    "h17_short",
    "h18_short",
    "h19_short",
    "h21_short",
    "h22_short",
    "h23_short",
 ]
for f in folders:
    new_file_name = f + ".ipynb"
    shutil.copy(original_file, new_file_name)